In [198]:
import pandas as pd
import seaborn as sns
import geopandas
import folium
import matplotlib.pyplot as plt
from shapely.geometry import Point


#print("All libraries imported. ")

In [202]:
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

# **Skateparks in Malmö**
*5 skateparks in Malmö.*

In [203]:
stapelbäddsparken_coordinates = [55.613781,12.983913]
#stapelbäddsparken_coordinates[1]

In [204]:
# Skatepark coordinates, stored as dictionary

coordinates_dictionary = {
    (55.613781,12.983913): 'Stapelbaddsparken',
    (55.571860, 12.976309): 'Kroksbacks Skatepark',
    (55.5920945, 13.0127095): 'Folkets Park',
    (55.574826, 12.908757): 'Sibbarp Skatepark',
    (55.541088, 13.103231): 'Oxie Skatepark'
}

In [205]:
map = folium.Map(width=1000, height=400, location = [55.614709,13.004293], tiles = "OpenStreetMap", zoom_start = 11.3)

# Looping values with keys with items() method and adding to map
for key, value in coordinates_dictionary.items():
    #print(key, value)
    folium.Marker( key, popup="Skatepark: " + str(value), icon=folium.Icon(color='black', 
                                                                           icon_color='white', 
                                                                           icon='angellist', angle=0, 
                                                                           prefix='fa') ).add_to(map) 
    
# Display map
map